# Building an Agent using LangChain

By themselves, language models can't take actions - they just output text. A big use case for `LangChain` is creating agents. `Agents` are systems that use `LLMs` as reasoning engines to determine which actions to take and the inputs necessary to perform the action. After executing actions, the results can be fed back into the `LLM` to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via `tool-calling`.

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

## Prerequisites

* Azure OpenAI GPT model
* Account in Tavily: [https://tavily.com/](https://tavily.com/)

To install `LangChain` run:

In [ ]:
%pip install -U langchain-community langgraph tavily-python --quiet

Note: you may need to restart the kernel to use updated packages.


Let's setup the API Key for Tavily as environment variable to be used by `TavilySearchResults`.

In [ ]:
import os

os.environ["TAVILY_API_KEY"] = "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"

We first need to create the tools we want to use. Our main tool of choice will be `Tavily` - a search engine. We have a built-in tool in `LangChain` to easily use `Tavily` search engine as tool.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1740715959, 'localtime': '2025-02-27 20:12'}, 'current': {'last_updated_epoch': 1740715200, 'last_updated': '2025-02-27 20:00', 'temp_c': 16.1, 'temp_f': 61.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 2.7, 'wind_kph': 4.3, 'wind_degree': 341, 'wind_dir': 'NNW', 'pressure_mb': 1016.0, 'pressure_in': 30.0, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 67, 'cloud': 50, 'feelslike_c': 16.1, 'feelslike_f': 61.0, 'windchill_c': 15.0, 'windchill_f': 59.0, 'heatindex_c': 15.1, 'heatindex_f': 59.2, 'dewpoint_c': 12.0, 'dewpoint_f': 53.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 5.6, 'gust_kph': 9.1}}"}, {'url': 'https://weathershogun.com/weather/u

## Using Language Models

Next, let's learn how to use a language model to call tools. LangChain supports many different language models that you can use interchangably.

In [21]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os

if os.path.exists(".env"):
    load_dotenv(override=True)

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

You can call the language model by passing in a list of messages. By default, the response is a content string.

In [22]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! 😊 How can I assist you today?'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use `.bind_tools` to give the language model knowledge of these tools.

In [23]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the content field as well as the `tool_calls` field.

In [24]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today? 😊
ToolCalls: []


Now, let's try calling it with some input that would expect a tool to be called.

In [29]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Tunis?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Tunis'}, 'id': 'call_AP0hrCkSgmlPGkraeuDUTN58', 'type': 'tool_call'}]


We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.

## Create the agent

Now that we have defined the tools and the `LLM`, we can create the agent. We will be using `LangGraph` to construct the agent. Currently, we are using a high level interface to construct the agent, but the nice thing about `LangGraph` is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.

Now, we can initialize the agent with the `LLM` and the tools.

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_react_agent` will call `.bind_tools` for us under the hood.

In [26]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run the agent

We can now run the agent with a few queries! Note that for now, these are all stateless queries (it won't remember previous interactions). Note that the agent will return the final state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:

In [27]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='af5946de-bbba-4b42-8b36-656f9388980b'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'h

Let's now try it out on an example where it should be invoking the tool

In [30]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Tunis?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in Tunis?', additional_kwargs={}, response_metadata={}, id='c0db1d45-9f18-4c35-aff2-497d3fb5d97f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5L3pLGmm4Iz3DYh3G1xU859x', 'function': {'arguments': '{"query":"current weather in Tunis"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 86, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexua

## Streaming Messages

We've seen how the agent can be called with .invoke to get a final response. If the agent executes multiple steps, this may take a while. To show intermediate progress, we can stream back messages as they occur.

In [31]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Paris?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in Paris?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_3f7Ug6QioKx0F5xE9HpTsQx9)
 Call ID: call_3f7Ug6QioKx0F5xE9HpTsQx9
  Args:
    query: current weather in Paris
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Paris', 'region': 'Ile-de-France', 'country': 'France', 'lat': 48.8667, 'lon': 2.3333, 'tz_id': 'Europe/Paris', 'localtime_epoch': 1740723519, 'localtime': '2025-02-28 07:18'}, 'current': {'last_updated_epoch': 1740723300, 'last_updated': '2025-02-28 07:15', 'temp_c': 5.1, 'temp_f': 41.2, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 3.6, 'wind_kph': 5.8, 'wind_degree': 352,

## More resources

https://python.langchain.com/docs/tutorials/agents/